In [20]:
# Import Dependencies
import requests
import json
from config import api_key
import datetime

In [21]:
####Project Skeleton v1 (Starbucks v1_04.15)###

# URL for GET requests to retrieve historical hospitalization data
url = "https://api.covidactnow.org/v2/county/TX.timeseries.json?"

# Date range for March 2021
start_date = "2021-03-01"
end_date = "2021-03-03"  # Adjusted to cover the full month of March 2021

# Query URL with date range and API key
query_url = f"{url}apiKey={api_key}&start={start_date}&end={end_date}"

# Function to recursively collect field names from a JSON object/dictionary (explore JSON object)
# Collect the path to every field - track field's full path
def collect_fields(json_obj, fields_set, parent_key=''):
    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            # Construct a "path" for nested fields
            new_key = f"{parent_key}.{key}" if parent_key else key
            fields_set.add(new_key)
            collect_fields(value, fields_set, new_key)
    elif isinstance(json_obj, list):
        for item in json_obj:
            collect_fields(item, fields_set, parent_key)

# Fetch data from the URL used to query API, parse as JSON, and extract fields
# Add a try-except block around the HTTP request and JSON parsing logic
def fetch_data_and_fields(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError exception for bad responses
        response_json = response.json()

        fields_set = set()
        collect_fields(response_json, fields_set)

        return list(fields_set)
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return []


In [22]:
# Make a GET request to retrieve the historical data
response = requests.get(query_url)
covid_data = response.json()

In [23]:
# Ensure covid_data is a list of dictionaries; each dictionary represents a county
vaccinations_march = []
for county_data in covid_data:
    county_name = county_data["county"]
    state_name = county_data["state"]
    # Check if 'actualsTimeseries' is a list and iterate through it
    for date_entry in county_data["actualsTimeseries"]:
        date = date_entry.get("date")
        # Filter for March 2023 entries
        if date.startswith("2021-03") or date.startswith("2022-03") or date.startswith("2023-03"):
            # Assuming 'vaccinationsInitiated' data exists and is structured as needed
            initvaccinations = date_entry.get("vaccinationsInitiated")
            complvaccinations = date_entry.get("vaccinationsCompleted")
            additionalvaccinations = date_entry.get("vaccinationsAdditionalDose")
            boostervaccinations = date_entry.get("vaccinationsFall2022BivalentBooster")
            # Append a new entry for each date in March 2023
            if initvaccinations is not None or complvaccinations is not None:
                vaccinations_march.append({
                    "State": state_name,
                    "County": county_name,
                    "Date": date,
                    "Initiated Vaccinations": initvaccinations,
                    "Completed Vaccinations": complvaccinations,
                    "Additional Dose": additionalvaccinations,
                    "Fall 2022 Booster": boostervaccinations
                })

# # Printing the number of different fields
print(f"Total vaccination entries for March: {len(vaccinations_march)}\n")

# Adjusted print to include date
for data in vaccinations_march:
    print(f"State: {data['State']}, County: {data['County']}, Date: {data['Date']}, Initiated Vaccinations: {data['Initiated Vaccinations']}, Completed Vaccinations: {data['Completed Vaccinations']}, Additional Dose: {data['Additional Dose']}, Fall 2022 Booster: {data['Fall 2022 Booster']}")

Total vaccination entries for March: 9098

State: TX, County: Anderson County, Date: 2022-03-01, Initiated Vaccinations: 25593, Completed Vaccinations: 22544, Additional Dose: 6948, Fall 2022 Booster: None
State: TX, County: Anderson County, Date: 2022-03-02, Initiated Vaccinations: 25589, Completed Vaccinations: 22551, Additional Dose: 6956, Fall 2022 Booster: None
State: TX, County: Anderson County, Date: 2022-03-03, Initiated Vaccinations: 25586, Completed Vaccinations: 22552, Additional Dose: 6964, Fall 2022 Booster: None
State: TX, County: Anderson County, Date: 2022-03-04, Initiated Vaccinations: 25587, Completed Vaccinations: 22553, Additional Dose: 7006, Fall 2022 Booster: None
State: TX, County: Anderson County, Date: 2022-03-05, Initiated Vaccinations: 25589, Completed Vaccinations: 22562, Additional Dose: 7015, Fall 2022 Booster: None
State: TX, County: Anderson County, Date: 2022-03-06, Initiated Vaccinations: 25589, Completed Vaccinations: 22562, Additional Dose: 7015, Fal

In [24]:
import pandas as pd
vaccine_df = pd.DataFrame(vaccinations_march)

vaccine_df

,State,County,Date,Initiated Vaccinations,Completed Vaccinations,Additional Dose,Fall 2022 Booster
0,TX,Anderson County,2022-03-01,25593,22544,6948,NaN
1,TX,Anderson County,2022-03-02,25589,22551,6956,NaN
2,TX,Anderson County,2022-03-03,25586,22552,6964,NaN
3,TX,Anderson County,2022-03-04,25587,22553,7006,NaN
4,TX,Anderson County,2022-03-05,25589,22562,7015,NaN
...,...,...,...,...,...,...,...
9093,TX,Zavala County,2023-03-01,7288,6241,2382,805.0
9094,TX,Zavala County,2023-03-08,7295,6246,2390,829.0
9095,TX,Zavala County,2023-03-15,7296,6248,2397,841.0
9096,TX,Zavala County,2023-03-22,7294,6250,2401,853.0


In [25]:
# Ensure covid_data is a list of dictionaries; each dictionary represents a county
total_vaccinations_march = []
for county_data in covid_data:
    county_name = county_data["county"]
    # Check if 'actualsTimeseries' is a list and iterate through it
    for date_entry in county_data["actualsTimeseries"]:
        date = date_entry.get("date")
        # Filter for March 2023 entries
        if date.startswith("2021-03") or date.startswith("2022-03") or date.startswith("2023-03"):
            # Assuming 'vaccinationsInitiated' data exists and is structured as needed
            complvaccinations = date_entry.get("vaccinationsCompleted")
            # Append a new entry for each date in March 2023
            if initvaccinations is not None or complvaccinations is not None:
                total_vaccinations_march.append({
                    "County": county_name,
                    "Date": date,
                    "Completed Vaccinations": complvaccinations
                })

# # Printing the number of different fields
print(f"Total vaccination entries for March: {len(total_vaccinations_march)}\n")

# Adjusted print to include date
for data in total_vaccinations_march:
    print(f"County: {data['County']}, Date: {data['Date']}, Completed Vaccinations: {data['Completed Vaccinations']}")

Total vaccination entries for March: 9098

County: Anderson County, Date: 2022-03-01, Completed Vaccinations: 22544
County: Anderson County, Date: 2022-03-02, Completed Vaccinations: 22551
County: Anderson County, Date: 2022-03-03, Completed Vaccinations: 22552
County: Anderson County, Date: 2022-03-04, Completed Vaccinations: 22553
County: Anderson County, Date: 2022-03-05, Completed Vaccinations: 22562
County: Anderson County, Date: 2022-03-06, Completed Vaccinations: 22562
County: Anderson County, Date: 2022-03-07, Completed Vaccinations: 22571
County: Anderson County, Date: 2022-03-08, Completed Vaccinations: 22574
County: Anderson County, Date: 2022-03-09, Completed Vaccinations: 22586
County: Anderson County, Date: 2022-03-10, Completed Vaccinations: 22593
County: Anderson County, Date: 2022-03-11, Completed Vaccinations: 22584
County: Anderson County, Date: 2022-03-12, Completed Vaccinations: 22590
County: Anderson County, Date: 2022-03-13, Completed Vaccinations: 22592
County: 

In [33]:
# response = requests.get(query_url)
# covid_data = response.json()

# Ensure covid_data is a list of dictionaries; each dictionary represents a county
county_vaccinations = {}  # Dictionary to store total new cases by county
for county_data in covid_data:
    county_name = county_data["county"]
    # Initialize the total new cases for the county if it's not already in the dictionary
    if county_name not in county_vaccinations:
        county_vaccinations[county_name] = 0
    # Check if 'actualsTimeseries' is a list and iterate through it
    for date_entry in county_data["actualsTimeseries"]:
        date = date_entry.get("date")
        # Filter for March 2021 entries
        if date.startswith("2022-03"):
            # Assuming 'hospitalBeds' data exists and is structured as needed
            vaccinations = date_entry.get("vaccinationsCompleted")
            if vaccinations is not None and vaccinations != 0:  # Check if new cases are not None and not equal to 0
                # Add new cases to the total for the county
                county_vaccinations[county_name] += vaccinations

# Printing vaccination data by county
print(f"Total counties used for data collection in March 2022: {len(county_vaccinations)}\n")
for county, total_cases in county_vaccinations.items():
    print(f"County: {county}, Total Vaccinations: {total_cases}")

Total counties used for data collection in March 2022: 254

County: Anderson County, Total Vaccinations: 701645
County: Andrews County, Total Vaccinations: 254614
County: Angelina County, Total Vaccinations: 1254461
County: Aransas County, Total Vaccinations: 413582
County: Archer County, Total Vaccinations: 158375
County: Armstrong County, Total Vaccinations: 23858
County: Atascosa County, Total Vaccinations: 820178
County: Austin County, Total Vaccinations: 442010
County: Bailey County, Total Vaccinations: 83704
County: Bandera County, Total Vaccinations: 361423
County: Bastrop County, Total Vaccinations: 1572430
County: Baylor County, Total Vaccinations: 44509
County: Bee County, Total Vaccinations: 503814
County: Bell County, Total Vaccinations: 6699167
County: Bexar County, Total Vaccinations: 42707112
County: Blanco County, Total Vaccinations: 209228
County: Borden County, Total Vaccinations: 6992
County: Bosque County, Total Vaccinations: 267640
County: Bowie County, Total Vacci

In [32]:
# Ensure covid_data is a list of dictionaries; each dictionary represents a county
county_vaccinations = {}  # Dictionary to store total new cases by county
for county_data in covid_data:
    county_name = county_data["county"]
    # Initialize the total new cases for the county if it's not already in the dictionary
    if county_name not in county_vaccinations:
        county_vaccinations[county_name] = 0
    # Check if 'actualsTimeseries' is a list and iterate through it
    for date_entry in county_data["actualsTimeseries"]:
        date = date_entry.get("date")
        # Filter for March 2021 entries
        if date.startswith("2023-03"):
            # Assuming 'hospitalBeds' data exists and is structured as needed
            vaccinations = date_entry.get("vaccinationsCompleted")
            if vaccinations is not None and vaccinations != 0:  # Check if new cases are not None and not equal to 0
                # Add new cases to the total for the county
                county_vaccinations[county_name] += vaccinations

# Printing vaccination data by county
print(f"Total counties used for data collection in March 2023: {len(county_vaccinations)}\n")
for county, total_cases in county_vaccinations.items():
    print(f"County: {county}, Total Vaccinations: {total_cases}")

Total counties used for data collection in March 2023: 254

County: Anderson County, Total Vaccinations: 116960
County: Andrews County, Total Vaccinations: 42328
County: Angelina County, Total Vaccinations: 206098
County: Aransas County, Total Vaccinations: 68293
County: Archer County, Total Vaccinations: 25791
County: Armstrong County, Total Vaccinations: 3800
County: Atascosa County, Total Vaccinations: 136997
County: Austin County, Total Vaccinations: 72206
County: Bailey County, Total Vaccinations: 13617
County: Bandera County, Total Vaccinations: 60315
County: Bastrop County, Total Vaccinations: 265435
County: Baylor County, Total Vaccinations: 7179
County: Bee County, Total Vaccinations: 84690
County: Bell County, Total Vaccinations: 1143094
County: Bexar County, Total Vaccinations: 7281904
County: Blanco County, Total Vaccinations: 34620
County: Borden County, Total Vaccinations: 994
County: Bosque County, Total Vaccinations: 42519
County: Bowie County, Total Vaccinations: 18963